In [173]:
import h5py
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random
# from tqdm.auto import tqdm
# from tqdm.notebook import tqdm
from SVWN3 import f_svwn3
from SVWN3_njit import f_svwn3_njit
import csv
import copy
import random
from NN_models import NN_2_256, NN_8_256, NN_8_64

In [174]:
def ref(x, y):
    ''' 
    returns reference energies for points of a reaction grid from Reference_data.csv
    '''
    hartree2kcal = 627.5095
    with open("Reference_data.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        k = 1
        if y == 391:
            k = hartree2kcal
        ref = []
        for n, i in enumerate(ref_file):
            if x <= n + 1 <= y:
                ref.append((i[0], float(i[2]) * k))

        return ref

def load_ref_energies():
    '''Returns {db_name: [equation, energy]}'''
    ref_e = { # Получение референсных энергий
        "MGAE109":ref(8, 116),
        "IP13":ref(155, 167),
        "EA13":ref(180, 192),
        "PA8":ref(195, 202),
        "DBH76":ref(251, 288) + ref(291, 328),
        "NCCE31":ref(331, 361),
        "ABDE4":ref(206, 209),
        # "AE17":ref(375, 391),
        "pTC13":ref(232, 234) + ref(237, 241) + ref(244, 248)
        } 
    return ref_e

def load_component_names():
    '''
    Returns {db_name: {id: {'Components': [...], 'Coefficients: [...]'
                                }
                            }
                        }
     which is a dictionary with Components and Coefficients data about all reactions
    '''
    with open("total_dataframe_sorted_final.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        ref = dict()
        current_database = None
        
        for n, line in enumerate(ref_file):
            line = np.array(line)
            if n == 0:
                components = np.array(line)
            else:
                reaction_id = int(line[0])
                reaction_database = line[1]
                reaction_component_num = np.nonzero(list(map(float, line[2:])))[0] + 2
                if reaction_database in ref:
                    ref[reaction_database][reaction_id] = {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}
                else: 
                    ref[reaction_database] = {reaction_id: {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}}
        return ref

In [175]:
def get_compounds_coefs_energy_v2(reactions, energies):
    '''Returns {id: 
                    {'Components': [...], 'Coefficients: [...]', 'Energy: float', Database: str
                                }
                            }
    which is a dictionaty from load_component_names with Energy information added
    '''
    data_final = dict()
    i = 0
    databases = load_ref_energies().keys()
    for database in databases:
        data = reactions[database]
        for reaction in data:
            data[reaction]['Energy'] = energies[database][reaction][1]
            data_final[i] = {'Database': database,
                         'Components': reactions[database][reaction]['Components'].astype(object),
                         'Coefficients': reactions[database][reaction]['Coefficients'].astype(np.float64),
                         'Energy': energies[database][reaction][1]
            
        }
            i += 1
        
    return data_final

In [176]:
def get_h5_names(reaction):
    '''reaction must be from the function get_compounds_coefs_energy_v2'''
    database_match = {
        'MGAE109': 'mgae109',
        'IP13': 'ip13',
        'EA13': 'ea13',
        'PA8': 'pa8',
        'DBH76': 'ntbh38',
        'NCCE31': 'ncce31',
        'ABDE4': 'abde4',
        'AE17': 'ae17',
        'pTC13': 'ptc13'
    }
    names = []
    for elem in reaction['Components']:
        database = database_match[reaction['Database']]
        names.append(f'{elem}.h5')
    return names

In [252]:
def add_reaction_info_from_h5(reaction):
    '''
    reaction must be from get_compounds_coefs_energy_v2
    returns merged descriptos array X, integration weights, 
    a and b densities and indexes for backsplitting
    
    Adds the following information to the reaction dict using h5 files from the dataset:
    Grid : np.array with grid descriptors
    Weights : list with integration weights of grid points
    Densities : np.array with alpha and beta densities data for grid points
    HF_energies : list of Total HF energy (T+V) which needs to be added to E_xc
    backsplit_ind: list of indexes where we concatenate molecules' grids
    '''
    X = np.array([]) 
    backsplit_ind = []
    HF_energies = np.array([])
    for component_filename in get_h5_names(reaction):
        with h5py.File(f'data/{component_filename}', "r") as f:
            HF_energies = np.append(HF_energies, f["ener"][:][0])
            X_raw = np.array(f["grid"][:])
            if len(X) == 0:
                X = X_raw[:, 3:-1]
            else:
                X = np.vstack((X, X_raw[:, 3:-1]))
            backsplit_ind.append(len(X))
    densities = X[:, 1:3]
    weights = X[:,0]
    X = X[:, 1:]

    labels = ['Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    values = [X, weights, densities, HF_energies, backsplit_ind]
    for label, value in zip(labels, values):
        reaction[label] = value

    return reaction

In [253]:
def make_reactions_dict():
    '''
    Returns a dict like {reaction_id: {*reaction info}} with all info available listed below:
    ['Database', 'Components', 'Coefficients', 'Energy', 'Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    '''
    data = get_compounds_coefs_energy_v2(load_component_names(), load_ref_energies())
    for i in data.keys():
        data[i] = add_reaction_info_from_h5(data[i])

    return data
    

In [254]:
def get_local_energies(reaction, constants):
    calc_reaction_data = {}
    densities = reaction['Densities']
    local_energies = torch.Tensor(list(map(f_svwn3, densities, constants)), requires_grad=True)
    calc_reaction_data['Local_energies'] = local_energies
    return calc_reaction_data


def add_calc_reaction_data(reaction, calc_reaction_data):
    calc_reaction_data['Weights'] = reaction['Weights']
    calc_reaction_data['Densities'] = reaction['Densities']
    return calc_reaction_data

def backsplit(reaction, calc_reaction_data):
    backsplit_ind = reaction['backsplit_ind']
    splitted_data = dict()
    stop = 0

    for i, component in enumerate(reaction['Components']):
        splitted_data[component] = dict()
        start = stop
        stop = backsplit_ind[i]
        for elem in ('Local_energies', 'Weights', 'Densities'):
            splitted_data[component][elem] = calc_reaction_data[elem][start:stop]
    return splitted_data


def integration(reaction, splitted_calc_reaction_data):
    molecule_energies = dict()
    for i, component in enumerate(reaction['Components']):
        molecule_energies[component] = np.sum(splitted_calc_reaction_data[component]['Local_energies'] \
                                              * (splitted_calc_reaction_data[component]['Densities'][:,0] \
                                              + splitted_calc_reaction_data[component]['Densities'][:,1]) \
                                              * (splitted_calc_reaction_data[component]['Weights'])) \
                                              + reaction['HF_energies'][i]
    return molecule_energies


def get_energy_reaction(reaction, molecule_energies):
    hartree2kcal = 627.5095
    s = 0
    for coef, ener in zip(reaction['Coefficients'], molecule_energies.values()):
        s += coef*ener
    reaction_energy_kcal = s * hartree2kcal
    return reaction_energy_kcal


def calculate_reaction_energy(reaction, constants):
    calc_reaction_data = get_local_energies(reaction, constants)
    calc_reaction_data = add_calc_reaction_data(reaction, calc_reaction_data)

    splitted_calc_reaction_data = backsplit(reaction, calc_reaction_data)

    molecule_energies = integration(reaction, splitted_calc_reaction_data)
    
    reaction_energy_kcal = get_energy_reaction(reaction, molecule_energies)

    return reaction_energy_kcal

In [255]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [256]:
data = make_reactions_dict()

In [245]:
data[73] # take the easiest reaction H2 = 2H

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype=object),
 'Coefficients': array([ 2., -1.]),
 'Energy': 109.49,
 'Grid': tensor([[3.1106e-01, 0.0000e+00, 3.6975e-09,  ..., 0.0000e+00, 1.4859e-09,
          0.0000e+00],
         [3.1106e-01, 0.0000e+00, 3.1224e-07,  ..., 0.0000e+00, 1.2547e-07,
          0.0000e+00],
         [3.1106e-01, 0.0000e+00, 4.1824e-06,  ..., 0.0000e+00, 1.6807e-06,
          0.0000e+00],
         ...,
         [3.3523e-04, 3.3523e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0039e-04,
          2.0039e-04],
         [4.5250e-04, 4.5250e-04, 9.8983e-07,  ..., 9.8983e-07, 2.7344e-04,
          2.7344e-04],
         [3.3523e-04, 3.3523e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0039e-04,
          2.0039e-04]], dtype=torch.float64),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02], dtype=torch.float64),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.

In [246]:
# tensor([3.1106e-01, 0.0000e+00, 3.6975e-09, 0.0000e+00, 0.0000e+00, 1.4859e-09,
        # 0.0000e+00], dtype=torch.float64)

In [247]:
def train_split(data, test_size, shuffle=False):
    if shuffle:
        keys = list(data.keys())
        random.shuffle(keys)
        for i in keys:
            data[keys[i]] = data[i]

    train, test = dict(), dict()
    border = round(len(data.keys()) * (1 - test_size))
    for i in range(len(data.keys())):
        if i <= border:
            train[i] = data[i]
        else:
            test[i] = data[i]
    return train, test


In [248]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())

test_size = 0.2

data_train, data_test = train_split(copy.deepcopy(data), 0.2, True)


train_set = Dataset(data=data_train)
# train_dataloader = torch.utils.data.DataLoader(train_set, 
#                                                batch_size=1)

test_set = Dataset(data=data_test)
# test_dataloader = torch.utils.data.DataLoader(test_set, 
#                                               batch_size=1)

In [250]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')

model = NN_8_256().to(device)
model.load_state_dict(torch.load('predoptimized_2.param'))

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.L1Loss()

In [251]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20):
    train_loss_mae = []
    test_loss_mae = []


    for epoch in range(n_epochs):
        print(epoch+1)
        # train
        model.train()
        # progress_bar = tqdm(train_loader)

        train_mae_losses_per_epoch = []
        for X_batch, y_batch in train_loader:
            X_batch_grid, y_batch = torch.Tensor(X_batch['Grid']).to(device), torch.Tensor(np.array([y_batch])).to(device)
            predictions = torch.expm1(model(X_batch_grid))
            
            reaction_energy = calculate_reaction_energy(X_batch, predictions)
            loss = criterion(np.array(reaction_energy), np.array([y_batch]))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_mae_losses_per_epoch.append(loss.item())
            print(X_batch['Components'], loss.item())
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        #test
        model.eval()
        test_mae_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                preds = np.expm1(model(X_batch_grid))
                reaction_energy = calculate_reaction_energy(X_batch, predictions)
                loss = criterion(np.array(reaction_energy), np.array(y_batch))
                test_mae_losses_per_epoch.append(loss.item())
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))
        print(f'train MAE Loss = {train_loss_mae[epoch]:.8f}')
        print(f'test MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds[0].cpu().detach().numpy()

In [223]:
train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds = train(model, criterion, optimizer, 
                                                                            train_set,
                                                                            test_set, n_epochs=20)

1


TypeError: expected TensorOptions(dtype=float, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=double, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))

In [257]:
reaction = data[73]
reaction

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype=object),
 'Coefficients': array([ 2., -1.]),
 'Energy': 109.49,
 'Grid': array([[3.11061027e-01, 0.00000000e+00, 3.69753029e-09, ...,
         0.00000000e+00, 1.48585405e-09, 0.00000000e+00],
        [3.11061017e-01, 0.00000000e+00, 3.12235408e-07, ...,
         0.00000000e+00, 1.25471930e-07, 0.00000000e+00],
        [3.11060900e-01, 0.00000000e+00, 4.18243025e-06, ...,
         0.00000000e+00, 1.68071198e-06, 0.00000000e+00],
        ...,
        [3.35234627e-04, 3.35234627e-04, 5.37413372e-07, ...,
         5.37413372e-07, 2.00387031e-04, 2.00387031e-04],
        [4.52496235e-04, 4.52496235e-04, 9.89827916e-07, ...,
         9.89827916e-07, 2.73435401e-04, 2.73435401e-04],
        [3.35234627e-04, 3.35234627e-04, 5.37413372e-07, ...,
         5.37413372e-07, 2.00387031e-04, 2.00387031e-04]]),
 'Weights': array([2.57172640e-17, 9.97800968e-15, 3.26110049e-13, ...,
        2.60546656e-02, 2.26799037e-02, 2

In [67]:
X_batch = reaction
y_batch = reaction['Energy']
X_batch_grid, y_batch = torch.Tensor(X_batch['Grid']).to(device), torch.Tensor(np.array([y_batch])).to(device)
predictions = torch.expm1(model(X_batch_grid))

In [72]:
reaction['Grid']

tensor([[2.7084e-01, 0.0000e+00, 3.6975e-09,  ..., 0.0000e+00, 1.4859e-09,
         0.0000e+00],
        [2.7084e-01, 0.0000e+00, 3.1224e-07,  ..., 0.0000e+00, 1.2547e-07,
         0.0000e+00],
        [2.7084e-01, 0.0000e+00, 4.1824e-06,  ..., 0.0000e+00, 1.6807e-06,
         0.0000e+00],
        ...,
        [3.3518e-04, 3.3518e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0037e-04,
         2.0037e-04],
        [4.5239e-04, 4.5239e-04, 9.8983e-07,  ..., 9.8983e-07, 2.7340e-04,
         2.7340e-04],
        [3.3518e-04, 3.3518e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0037e-04,
         2.0037e-04]])

In [266]:
def get_local_energies(reaction, constants):
    calc_reaction_data = {}
    densities = reaction['Densities']
    # local_energies = [f_svwn3(x, y) for x, y in zip(densities[:1000], constants[:1000])]
    local_energies = np.array([f_svwn3_njit(x, y) for x, y in zip(densities[:1000], constants[:1000])])
    calc_reaction_data['Local_energies'] = local_energies
    return calc_reaction_data

In [267]:
true_const = np.array([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*np.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])

In [268]:
%%time
local_energies = get_local_energies(reaction, true_const) # 1k of 100k grid points for 2H = H2 system

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(float64, Literal[int](20))
 
There are 22 candidate implementations:
   - Of which 22 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(float64, int64)':
    No match.

During: typing of intrinsic-call at /tmp/job-895719/ipykernel_185596/3635234700.py (97)
During: typing of static-get-item at /tmp/job-895719/ipykernel_185596/3635234700.py (97)

File "../../../tmp/job-895719/ipykernel_185596/3635234700.py", line 97:
<source missing, REPL/exec in use?>

During: resolving callee type: type(CPUDispatcher(<function f_lda_x at 0x2ae52555d5a0>))
During: typing of call at /tmp/job-895719/ipykernel_185596/3635234700.py (106)

During: resolving cal

In [234]:
local_energies['Local_energies'][:50]

[tensor(nan, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6630, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6630, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6630, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6630, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6629, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(nan, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(nan, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6630, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(nan, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.6631, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-0.66

In [235]:
import numpy as np
from numba import njit



# def constants_naming(c_arr):

#     labels = ['A_vwn', 'b_vwn', 'c_vwn', 'x0_vwn', 'A_rpa', 'b_rpa', 'c_rpa', 'x0_rpa', 'params_a_alpha']
#     values = [c_arr[0:2],c_arr[2:4],c_arr[4:6],c_arr[6:8],c_arr[8:11],c_arr[11:14],c_arr[14:17],c_arr[17:20], c_arr[20]]
    
#     return {labels[i]: values[i] for i in range(len(labels))}


#VWN
p_a_zeta_threshold = 1e-15

fpp_vwn = 4/(9*(2**(1/3) - 1))

@njit
def Q_vwn(b, c):
    return np.sqrt(4*c - b**2)

@njit
def f1_vwn(b, c):
    return 2*b/Q_vwn(b, c)

@njit
def f2_vwn(b, c, x0):
    return b*x0/(x0**2 + b*x0 + c)

@njit
def f3_vwn(b, c, x0):
    return 2*(2*x0 + b)/Q_vwn(b, c)

@njit
def fx_vwn(b, c, rs):
    return rs + b*np.sqrt(rs) + c

@njit
def opz_pow_n(z, n):
    if 1 + z <= p_a_zeta_threshold:
        return (p_a_zeta_threshold)^n
    else:
        return (1+z)**n

@njit
def f_aux(A, b, c, x0, rs):
    return A*(
    + np.log(rs/fx_vwn(b, c, rs))
    + (f1_vwn(b, c) - f2_vwn(b, c, x0)*f3_vwn(b, c, x0))
    * np.arctan(Q_vwn(b, c)/(2*np.sqrt(rs) + b))
    - f2_vwn(b, c, x0)*np.log((np.sqrt(rs) - x0)**2/fx_vwn(b, c, rs)))

@njit
def DMC(rs, z, c_arr):
    return f_aux(c_arr[0:2][1], c_arr[2:4][1], c_arr[4:6][1], c_arr[6:8][1], rs) \
    - f_aux(c_arr[0:2][0], c_arr[2:4][0], c_arr[4:6][0], c_arr[6:8][0], rs)

@njit
def DRPA(rs, z, c_arr):
    return f_aux(c_arr[8:11][1], c_arr[11:14][1], c_arr[14:17][1], c_arr[17:20][1], rs) \
    - f_aux(c_arr[8:11][0], c_arr[11:14][0], c_arr[14:17][0], c_arr[17:20][0], rs)

#VWN3

@njit
def f_zeta(z): # - power threshold
    return ((1 + z)**(4/3) + (1 - z)**(4/3) - 2)/(2**(4/3) - 2)

@njit
def f_vwn(rs, z, c_arr):
    return f_aux(c_arr[0:2][0], c_arr[2:4][0], c_arr[4:6][0], c_arr[6:8][0], rs) \
    + DMC(rs, z, c_arr)/DRPA(rs, z, c_arr)*f_aux(c_arr[8:11][2], c_arr[11:14][2], c_arr[14:17][2], c_arr[17:20][2], rs) \
    * f_zeta(z)*(1 - z**4)/fpp_vwn + DMC(rs, z, c_arr)*f_zeta(z)*z**4

@njit
def rs_z_calc(rho):
    rs = (3/((rho[0] + rho[1]) * (4 * np.pi))) ** (1/3)
    z = (rho[0] - rho[1]) / (rho[0] + rho[1])
    return rs, z

@njit
def f_vwn3(rho, c_arr):
    rs, z = rs_z_calc(rho)
    return f_vwn(rs, z, c_arr)


#SLATER

LDA_X_FACTOR = -3/8*(3/np.pi)**(1/3)*4**(2/3)
RS_FACTOR = (3/(4*np.pi))**(1/3)
DIMENSIONS = 3


@njit
def f_lda_x(rs, z, c_arr): # - screen_dens threshold
    return c_arr[20]*lda_x_spin(rs, z) + c_arr[20]*lda_x_spin(rs, -z)

@njit
def lda_x_spin(rs, z):
    return LDA_X_FACTOR*(z+1)**(1 + 1/DIMENSIONS)*2**(-1-1/DIMENSIONS)*(RS_FACTOR/rs)

@njit
def f_slater(rho, c_arr):
    rs, z = rs_z_calc(rho)
    return f_lda_x(rs, z, c_arr)

@njit
def f_svwn3_njit(rho, c_arr):
    '''rho.shape = (2, 1)'''
    return f_slater(rho, c_arr) + f_vwn3(rho, c_arr)

In [ ]:
f_svwn3(reaction['Densities'].to('cpu')[0], predictions.to('cpu')[0])

In [228]:
reaction['Densities'].to('cpu')[0]

tensor([0.3111, 0.0000])

In [100]:
predictions.to('cpu')[0]

tensor([ 3.1087e-02,  1.5544e-02,  3.7274e+00,  7.0604e+00,  1.2935e+01,
         1.8058e+01, -1.0498e-01, -3.2500e-01,  3.1098e-02,  1.5544e-02,
        -1.6889e-02,  1.3072e+01,  2.0123e+01,  1.0684e+00,  4.2719e+01,
         1.0158e+02,  1.1481e+01, -4.0928e-01, -7.4329e-01, -2.2835e-01,
         1.0000e+00], grad_fn=<SelectBackward0>)

In [91]:
local_energies['Local_energies']

[tensor(nan, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(-0.6629, grad_fn=<AddBackward0>),
 tensor(nan, grad_fn=<AddBackward0>),
 tensor(nan, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(nan, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6631, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(nan, grad_fn=<AddBackward0>),
 tensor(-0.6630, grad_fn=<AddBackward0>),
 tensor(-0.6629, grad_fn=<AddBackward0>),
 tensor(nan, grad_fn=<AddBackward0>),
 tensor(-0.6624, grad_fn=<AddBackward0>),
 tensor(-0.6631,

In [89]:
torch.cuda.memory_allocated()/1024**3 # GB

2.548874855041504

In [88]:
torch.cuda.empty_cache()